In [1]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.set_random_seed(10)
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'image.origin': 'lower',
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'axes.grid': False,
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 16, # fontsize for x and y labels (was 10)
    'axes.titlesize': 16,
    'font.size': 16, # was 10
    'legend.fontsize': 16, # was 10
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'text.usetex': True,
    'figure.figsize': [3.39, 2.10],
    'font.family': 'serif',
}
plt.rcParams.update(params)

In [3]:
# Generating training data that goes from initial condition location to PCA coefficient trajectory
num_modes=40
locs = np.load('../../SWE_Data/Data/Locations.npy')
pca_coeffs = np.load('../../SWE_Data/PCA_Coefficients_q1.npy')[0:num_modes,:]

coeff_scaler = MinMaxScaler()
pca_coeffs_scaled = np.transpose(coeff_scaler.fit_transform(np.transpose(pca_coeffs)))

In [4]:
num_sims = np.shape(locs)[0]
num_ivs = np.shape(locs)[1]
num_coeffs = np.shape(pca_coeffs)[0]
burn_in = 20

In [5]:
# Reshape
training_data_ip = np.zeros(shape=(num_sims,num_coeffs+num_ivs,burn_in),dtype='double')
training_data_op = np.zeros(shape=(num_sims,num_coeffs+num_ivs,500-burn_in),dtype='double')

for sim in range(num_sims):
    training_data_ip[sim,:-num_ivs,:] = pca_coeffs_scaled[:,500*sim:500*sim+burn_in]
    training_data_ip[sim,-num_ivs:,:] = locs[sim,:,None]
    training_data_op[sim,:-num_ivs,:] = pca_coeffs_scaled[:,500*sim+burn_in:500*(sim+1)]
    training_data_op[sim,-num_ivs:,:] = locs[sim,:,None]

In [6]:
from tensorflow.keras.layers import Input, Dense, Lambda, Add, LSTM, Dropout
from tensorflow.keras import optimizers, models, regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import plot_model

In [7]:
weights_filepath = 'NA_LSTM_P.h5'

def coeff_determination(y_pred, y_true): #Order of function inputs is important here        
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [8]:
lstm_inputs = Input(shape=(num_coeffs+num_ivs,burn_in),name='nat_inputs')
h1 = LSTM(195,return_sequences=True)(lstm_inputs)
h1 = Dropout(0.2)(h1,training=True)
h2 = LSTM(195,return_sequences=True)(h1)
h2 = Dropout(0.2)(h2,training=True)
lstm_outputs = Dense(500-burn_in,activation=None)(h2)

lstm_model = Model(inputs=lstm_inputs,outputs=lstm_outputs)
 
# design network
my_adam = optimizers.Adam(lr=0.00179731990744, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

checkpoint = ModelCheckpoint(weights_filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
earlystopping = EarlyStopping(monitor='loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
callbacks_list = [checkpoint,EarlyStoppingByLossVal()]

# fit network
lstm_model.compile(optimizer=my_adam,loss='mean_squared_error',metrics=[coeff_determination])    
lstm_model.summary()

W0127 14:40:33.884612 140466813642560 deprecation.py:506] From /home/rmlans/anaconda3/envs/deephyper_env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
nat_inputs (InputLayer)      [(None, 42, 20)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 42, 195)           168480    
_________________________________________________________________
dropout (Dropout)            (None, 42, 195)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 42, 195)           304980    
_________________________________________________________________
dropout_1 (Dropout)          (None, 42, 195)           0         
_________________________________________________________________
dense (Dense)                (None, 42, 480)           94080     
Total params: 567,540
Trainable params: 567,540
Non-trainable params: 0
_______________________________________________________

In [8]:
num_epochs = 5000
batch_size = 4
train_mode = False

if train_mode:
    train_history = lstm_model.fit(x=training_data_ip, y=training_data_op, epochs=num_epochs, batch_size=batch_size, callbacks=callbacks_list)
    np.save('NA_LSTM_P.npy',train_history.history['loss'])

W0121 19:42:51.270466 139773456013120 deprecation.py:323] From /home/rmlans/anaconda3/envs/deephyper_env/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5000
12/20 [=================>............] - ETA: 0s - loss: 0.2528 - coeff_determination: -134.6705
Epoch 00001: loss improved from inf to 0.24896, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 1s 50ms/sample - loss: 0.2490 - coeff_determination: -123.0971
Epoch 2/5000
12/20 [=================>............] - ETA: 0s - loss: 0.2206 - coeff_determination: -47.8942
Epoch 00002: loss improved from 0.24896 to 0.21088, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 13ms/sample - loss: 0.2109 - coeff_determination: -36.3195
Epoch 3/5000
12/20 [=================>............] - ETA: 0s - loss: 0.1600 - coeff_determination: -7.4376
Epoch 00003: loss improved from 0.21088 to 0.14780, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 13ms/sample - loss: 0.1478 - coeff_determination: -5.6305
Epoch 4/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.1008 - coeff_determination: -1.0994
Epoch 00004

12/20 [=================>............] - ETA: 0s - loss: 0.0222 - coeff_determination: -0.6568
Epoch 00029: loss improved from 0.02319 to 0.02206, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 11ms/sample - loss: 0.0221 - coeff_determination: -0.6751
Epoch 30/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0208 - coeff_determination: -0.5053
Epoch 00030: loss improved from 0.02206 to 0.02142, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 10ms/sample - loss: 0.0214 - coeff_determination: -0.6240
Epoch 31/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0212 - coeff_determination: -0.6849
Epoch 00031: loss improved from 0.02142 to 0.02081, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 11ms/sample - loss: 0.0208 - coeff_determination: -0.6614
Epoch 32/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0210 - coeff_determination: -0.7093
Epoch 00032: loss impro

Epoch 58/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0145 - coeff_determination: 0.1761
Epoch 00058: loss did not improve from 0.01427
20/20 [==============================] - 0s 14ms/sample - loss: 0.0143 - coeff_determination: 0.1660
Epoch 59/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0137 - coeff_determination: 0.2541
Epoch 00059: loss improved from 0.01427 to 0.01403, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 13ms/sample - loss: 0.0140 - coeff_determination: 0.2149
Epoch 60/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0140 - coeff_determination: 0.1466
Epoch 00060: loss improved from 0.01403 to 0.01395, saving model to NAT_LSTM_PT.h5
20/20 [==============================] - 0s 12ms/sample - loss: 0.0140 - coeff_determination: 0.1105
Epoch 61/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0135 - coeff_determination: 0.3005
Epoch 00061: loss improved from 0.01395 to 0.01357, 

Epoch 89/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0129 - coeff_determination: 0.3399
Epoch 00089: loss did not improve from 0.01276
20/20 [==============================] - 0s 8ms/sample - loss: 0.0128 - coeff_determination: 0.2577
Epoch 90/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0133 - coeff_determination: 0.2034
Epoch 00090: loss did not improve from 0.01276
20/20 [==============================] - 0s 8ms/sample - loss: 0.0132 - coeff_determination: 0.2176
Epoch 91/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0129 - coeff_determination: 0.2563
Epoch 00091: loss did not improve from 0.01276
20/20 [==============================] - 0s 8ms/sample - loss: 0.0130 - coeff_determination: 0.2651
Epoch 92/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0126 - coeff_determination: 0.1531
Epoch 00092: loss did not improve from 0.01276
20/20 [==============================] - 0s 8ms/sample - loss: 0.0129 - coeff_de

...

Epoch 4868/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9075
Epoch 04868: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0023 - coeff_determination: 0.9059
Epoch 4869/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8976
Epoch 04869: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0024 - coeff_determination: 0.9002
Epoch 4870/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8845
Epoch 04870: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0024 - coeff_determination: 0.8983
Epoch 4871/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9101
Epoch 04871: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.002

Epoch 4900/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9002
Epoch 04900: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0023 - coeff_determination: 0.9059
Epoch 4901/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8798
Epoch 04901: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0025 - coeff_determination: 0.8893
Epoch 4902/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0022 - coeff_determination: 0.9151
Epoch 04902: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.0022 - coeff_determination: 0.9029
Epoch 4903/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0022 - coeff_determination: 0.9139
Epoch 04903: loss did not improve from 0.00214
20/20 [==============================] - 0s 10ms/sample - loss: 0.002

Epoch 4932/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8878
Epoch 04932: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.0024 - coeff_determination: 0.8945
Epoch 4933/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8985
Epoch 04933: loss did not improve from 0.00214
20/20 [==============================] - 0s 12ms/sample - loss: 0.0025 - coeff_determination: 0.8990
Epoch 4934/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.9116
Epoch 04934: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.0024 - coeff_determination: 0.9084
Epoch 4935/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0024 - coeff_determination: 0.8982
Epoch 04935: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.002

Epoch 4964/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9073
Epoch 04964: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.0023 - coeff_determination: 0.9049
Epoch 4965/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.8894
Epoch 04965: loss did not improve from 0.00214
20/20 [==============================] - 0s 14ms/sample - loss: 0.0024 - coeff_determination: 0.8968
Epoch 4966/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9181
Epoch 04966: loss did not improve from 0.00214
20/20 [==============================] - 0s 11ms/sample - loss: 0.0023 - coeff_determination: 0.9010
Epoch 4967/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0025 - coeff_determination: 0.9031
Epoch 04967: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.002

Epoch 4996/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.8935
Epoch 04996: loss did not improve from 0.00214
20/20 [==============================] - 0s 12ms/sample - loss: 0.0023 - coeff_determination: 0.9020
Epoch 4997/5000
12/20 [=================>............] - ETA: 0s - loss: 0.0025 - coeff_determination: 0.9029
Epoch 04997: loss did not improve from 0.00214
20/20 [==============================] - 0s 12ms/sample - loss: 0.0025 - coeff_determination: 0.8906
Epoch 4998/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0023 - coeff_determination: 0.9055
Epoch 04998: loss did not improve from 0.00214
20/20 [==============================] - 0s 12ms/sample - loss: 0.0023 - coeff_determination: 0.9097
Epoch 4999/5000
16/20 [=======================>......] - ETA: 0s - loss: 0.0022 - coeff_determination: 0.9077
Epoch 04999: loss did not improve from 0.00214
20/20 [==============================] - 0s 13ms/sample - loss: 0.002

In [9]:
import matplotlib.pyplot as plt
lstm_model.load_weights(weights_filepath)

# Testing
filename = '../../SWE_Data/Data/snapshot_matrix_pod_test.npy'
test_data = np.load(filename)[0:64*64,:]
pca_vectors = np.load('../../SWE_Data/PCA_Vectors_q1.npy')[:,:num_modes]

true_pca_evol = coeff_scaler.transform(np.matmul(np.transpose(test_data),pca_vectors))
test_data = np.zeros(shape=(1,num_coeffs+num_ivs,500))
test_data[0,0:num_coeffs,:] = np.transpose(true_pca_evol[:,:])

test_data[0,-2,:] = -1.0/2.7
test_data[0,-1,:] = -1.0/4.0
viz = False

mse_val = 0.0
num_inference = 1000
pred_mean = np.zeros_like(test_data)
pred_pca_array = np.zeros(shape=(num_inference,np.shape(test_data)[1],np.shape(test_data)[2]))

from time import time

start_time = time()

for inference in range(num_inference):
    pred_pca = lstm_model.predict(test_data[:,:,:burn_in])
    pred_pca = np.concatenate((test_data[:,:,:burn_in],pred_pca),axis=-1)
    pred_pca_array[inference,:,:] = pred_pca[0,:,:]
    
    pred_mean = pred_mean + pred_pca
    mse_val = mse_val + np.sum((pred_pca-test_data)**2)
    
    if viz:
        
        fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(12,10))
        ax[0,0].plot(test_data[0,0,:],label='True')
        ax[0,0].plot(pred_pca[0,0,:],label='Predicted')
        ax[0,0].set_title('Mode 1')


        ax[1,0].plot(test_data[0,1,:],label='True')
        ax[1,0].plot(pred_pca[0,1,:],label='Predicted')
        ax[1,0].set_title('Mode 2')

        ax[0,1].plot(test_data[0,2,:],label='True')
        ax[0,1].plot(pred_pca[0,2,:],label='Predicted')
        ax[0,1].set_title('Mode 3')

        ax[1,1].plot(test_data[0,3,:],label='True')
        ax[1,1].plot(pred_pca[0,3,:],label='Predicted')
        ax[1,1].set_title('Mode 4')

        plt.tight_layout()
        plt.legend()
        plt.show()
        
        # Plotting some contours
        true_rb = np.transpose(coeff_scaler.inverse_transform(true_pca_evol))
        true_recon = np.matmul(pca_vectors,true_rb)[:,-1].reshape(64,64)

        pred_rb = np.transpose(pred_pca[0,:num_modes,:])
        pred_rb = np.transpose(coeff_scaler.inverse_transform(pred_rb))
        pred_recon = np.matmul(pca_vectors,pred_rb)[:,-1].reshape(64,64)


        fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(12,5))
        cx = ax[0].contourf(true_recon)
        ax[1].contourf(pred_recon)

        fig.colorbar(cx,ax=ax[0],fraction=0.046, pad=0.04)
        fig.colorbar(cx,ax=ax[1],fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()    
        
end_time = time()

In [11]:
print('Elapsed time per inference:',(end_time-start_time)/num_inference)

Elapsed time per inference: 0.016597886085510254


In [12]:
if train_mode:
    pred_mean = pred_mean/num_inference
    pred_sdev = np.sqrt(np.sum((pred_pca_array - pred_mean[0,:,:])**2,axis=0)/(num_inference-1))
    np.save('../Figures/NA_LSTM_P_Mean.npy',pred_mean)
    np.save('../Figures/NA_LSTM_P_SD.npy',pred_sdev)